In [7]:
class backtest():
    def __init__(self,data_out,price):
        self.data_out=data_out.dropna().iloc[::-1]#late to the last
        self.price=price.head(len(self.data_out)).iloc[::-1]
    def backtest(self,trade_size=100,ifshort=True):
        prev_x_qty = 0 #amount
        position = 0 #-1,0,1
        trade_size = trade_size
        signal = np.sign(self.data_out.Signal)#############Signal column is requuired in data_out
        self.data_out['buyOrsell']=signal
        qty_x = pd.Series(np.zeros(len(signal)),index=self.data_out.index) #money spend      
        if ifshort:
            for i in range(len(signal)) :
              if signal[i] == 1 and position == 0 :
                # buy the spread initially
                prev_x_qty = trade_size
                qty_x[i] = prev_x_qty
                position = 1

              if signal[i] == -1 and position == 0 :
                # sell the spread initially
                prev_x_qty =  trade_size
                qty_x[i] = -prev_x_qty
                position = -1

              if signal[i] == 1 and position == -1 :
                # we were short the spread and need to buy
                qty_x[i] = trade_size + prev_x_qty#新头寸+平旧头寸
                prev_x_qty = trade_size
                position = 1


              if signal[i] == -1 and position == 1 :
                # we are long the spread and need to sell
                qty_x[i] = -(trade_size + prev_x_qty)
                prev_x_qty = trade_size
                position = -1            
        else:
            for i in range(len(signal)) :
              if signal[i] == 1 and position == 0 :
                # buy the spread initially
                qty_x[i] = trade_size
                position = 1


              if signal[i] == -1 and position == 0 :
                # sell the spread initially
                qty_x[i] = 0
                position = 0

              if signal[i] == -1 and position == 1 :
                # we are long the spread and need to sell
                qty_x[i] = - trade_size 
                position = 0
            
        qty_x[len(qty_x)-1] = -sum(qty_x)#zero out
        self.data_out["qty_x"] = qty_x#对x交易行为序列
        
        cash_buy=[(np.sign(q)==1)*q*self.price[i] for i,q in enumerate(self.data_out.qty_x) ]
        cash_sell=[-((np.sign(q)==-1)*q*self.price[i]) for i,q in enumerate(self.data_out.qty_x) ]
        pos = np.cumsum(self.data_out.qty_x)
        cumulative_buy = np.cumsum(cash_buy)#现金支出
        cumulative_sell = np.cumsum(cash_sell)#现金收入
        equity = (cumulative_sell - cumulative_buy) + pos * self.price#realized P&L+unrealized P&L
        self.data_out['equity_curve']=equity
        
    def plot_curve(self):
        data = [go.Scatter(
                  x=self.data_out.equity_curve.index,
                  y=self.data_out.equity_curve.values)]
        return py.iplot(data)





In [8]:
%run macd.py
#return a variable a which contain ohlc data and has method macd

In [9]:
mb=backtest(a.macd(12,26,9),a.close)

In [ ]:
mb.backtest(trade_size=100,ifshort=True)
mb.plot_curve()

In [6]:
mb.data_out

,12,26,macd,9,Signal,buyOrsell,qty_x
2016-11-18,7.901526,7.452316,0.449210,0.513086,-0.063877,-1,-100
2016-11-21,7.993599,7.529923,0.463676,0.503204,-0.039528,-1,0
2016-11-22,8.157661,7.643262,0.514399,0.505443,0.008956,1,200
2016-11-23,8.314944,7.757094,0.557850,0.515925,0.041925,1,0
2016-11-24,8.428029,7.852865,0.575164,0.527773,0.047392,1,0
2016-11-25,8.576025,7.966727,0.609298,0.544078,0.065220,1,0
2016-11-28,8.845867,8.141784,0.704083,0.576079,0.128004,1,0
2016-11-29,9.203426,8.366096,0.837330,0.628329,0.209001,1,0
2016-11-30,9.482899,8.562682,0.920217,0.686707,0.233511,1,0
2016-12-01,9.674761,8.723224,0.951537,0.739673,0.211864,1,0
